step 1 & 2

In [37]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression , ElasticNet 
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np

data = fetch_california_housing()

X = data.data
y = data.target
feature_name = data.feature_names
print(feature_name)
X.shape
print(X.shape)
data

df = pd.DataFrame(X , columns= feature_name)
df["MedHouseVal"] = y

df


['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
(20640, 8)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [38]:
X_train , X_test, y_train, y_test = train_test_split(X , y , test_size=0.2 , random_state= 42)

Scaled = StandardScaler()
Scaled.fit(X_train)
X_train_scaled = Scaled.transform(X_train)
X_test_scaled = Scaled.transform(X_test)


#fit just train
#do not scale target(y doesnt need to be normalized)


step 3

In [39]:
lin_model = LinearRegression()
lin_model.fit(X_train_scaled, y_train)
y_test_pred = lin_model.predict(X_test_scaled)
y_train_pred = lin_model.predict(X_train_scaled)
r2_test =r2_score(y_test , y_test_pred)
r2_train = r2_score(y_train , y_train_pred)

overfitting_lin =r2_train - r2_test
overfitting_lin


0.03676348536424423

In [40]:
m = lin_model.coef_
df_m = pd.DataFrame(m , index=feature_name , columns= ["coef_lin"])
df_m.sort_values(by = "coef_lin" , ascending=False)



,coef_lin
MedInc,0.854383
AveBedrms,0.339259
HouseAge,0.122546
Population,-0.002308
AveOccup,-0.040829
AveRooms,-0.294410
Longitude,-0.869842
Latitude,-0.896929


step 4

In [41]:
elastic_model =ElasticNet(alpha=0.1 , l1_ratio=0.5)

elastic_model.fit(X_train_scaled, y_train)
y_train_pred_elas = elastic_model.predict(X_train_scaled)
r2_train_elas = r2_score(y_train, y_train_pred_elas)
y_test_pred_elas = elastic_model.predict(X_test_scaled)
r2_test_elas= r2_score(y_test , y_test_pred_elas)

overfitting_elastic =r2_train_elas - r2_test_elas
overfitting_elastic 

0.015997014664145937

In [42]:
mel = elastic_model.coef_
df_e = pd.DataFrame(mel , index=feature_name , columns= ["coef_elas"])
df_e.sort_values(by = "coef_elas" , ascending=False)

,coef_elas
MedInc,0.713818
HouseAge,0.137288
AveRooms,-0.000000
AveBedrms,0.000000
Population,0.000000
AveOccup,-0.000000
Longitude,-0.133250
Latitude,-0.175737


In [43]:
diff_df=pd.concat([df_m , df_e] , axis=1)
diff_df.columns = ["coef_lin", "coef_elas"]
diff_df["diff"] = diff_df["coef_lin"] - diff_df["coef_elas"]

diff_df

,coef_lin,coef_elas,diff
MedInc,0.854383,0.713818,0.140565
HouseAge,0.122546,0.137288,-0.014742
AveRooms,-0.294410,-0.000000,-0.294410
AveBedrms,0.339259,0.000000,0.339259
Population,-0.002308,0.000000,-0.002308
AveOccup,-0.040829,-0.000000,-0.040829
Latitude,-0.896929,-0.175737,-0.721192
Longitude,-0.869842,-0.133250,-0.736592


1
در Elastic Net، ویژگی‌هایی که به مدل کمک می‌کنند، ضرایب آن‌ها بزرگتر می‌شود.

ویژگی‌هایی که تاثیر کمی دارند یا بی‌فایده هستند، کاهش داده می‌شوند (حتی به صفر می‌رسند).

2
Elastic Net به دلیل داشتن regularization در برابر overfitting مقاوم‌تر است، زیرا در کاهش ضرایب ویژگی‌ها (shrinkage) موفق است.

Linear Regression بدون regularization می‌تواند به راحتی با داده‌های آموزشی تطابق پیدا کند و overfitting بیشتری داشته باشد.

3

مفهوم Shrinkage در Elastic Net:

Elastic Net ضرایب را کوچک‌تر می‌کند، به‌ویژه برای ویژگی‌هایی که تأثیر کمی بر پیش‌بینی‌ها دارند. این باعث می‌شود که مدل به خوبی generalize کند و از ویژگی‌های کمتری استفاده کند، که در نهایت می‌تواند overfitting را کاهش دهد.

Elastic Net از regularization برای shrinkage یا کوچک کردن ضرایب استفاده می‌کند. Shrinkage به این معنی است که Elastic Net سعی می‌کند ضرایب مدل را کاهش دهد تا از overfitting جلوگیری کند.

4

مدل Elastic Net به احتمال زیاد برای generalization بهتر است.

Generalization: توانایی مدل برای عمل کردن روی داده‌های جدید.

Overfitting: مدل به داده‌های آموزشی خیلی تطابق پیدا می‌کنه و نمی‌تونه روی داده‌های جدید خوب عمل کنه.

Regularization: تکنیک‌هایی برای جلوگیری از overfitting و کاهش پیچیدگی مدل.

Shrinkage: کوچک‌تر کردن ضرایب ویژگی‌ها برای کاهش پیچیدگی مدل و جلوگیری از overfitting.

step 6.2

Regularization چیست؟

Regularization در مدل‌های یادگیری ماشین به مجموعه‌ای از تکنیک‌ها گفته می‌شود که به منظور جلوگیری از overfitting و کم کردن پیچیدگی مدل به کار می‌روند. این تکنیک‌ها با افزودن یک جریمه به تابع هزینه (loss 
function) مدل، سعی می‌کنند که ضرایب ویژگی‌ها را کاهش دهند و از یادگیری ویژگی‌های غیرضروری جلوگیری کنند.

چطور Regularization باعث جلوگیری از Overfitting می‌شود؟

Regularization با افزودن یک مجازات به تابع هزینه باعث می‌شود که مدل از یادگیری ویژگی‌هایی که تأثیر کمی بر پیش‌بینی‌ها دارند جلوگیری کند. این کار معمولاً به این صورت انجام می‌شود که ضرایب ویژگی‌ها را محدود می‌کند.

انواع Regularization:

L1 Regularization (Lasso):

در این روش، مجموع مقادیر مطلق ضرایب به تابع هزینه افزوده می‌شود.

نتیجه این است که برخی از ضرایب ویژگی‌ها به صفر می‌رسند (یعنی ویژگی‌های غیرضروری حذف می‌شوند).

به این ترتیب، feature selection هم به صورت خودکار انجام می‌شود.

L2 Regularization (Ridge):

در این روش، مجموع مربعات ضرایب به تابع هزینه افزوده می‌شود.

این باعث می‌شود که ضرایب مدل کوچک‌تر شوند، ولی به صفر نمی‌رسند.

در نتیجه، مدل پیچیدگی کمتری دارد و احتمال overfitting کاهش می‌یابد.

Elastic Net:

این تکنیک ترکیبی از L1 و L2 Regularization است.

برای مدل‌هایی که هم از ویژگی‌های کمی (L1) و هم از ویژگی‌های زیاد (L2) استفاده می‌کنند، کاربرد دارد




Lasso مخفف Least Absolute Shrinkage and Selection Operator

همچنین در مشکلات high-dimensional که تعداد ویژگی‌ها بسیار زیاد است، L1 regularization کمک می‌کند تا ویژگی‌های غیرضروری حذف شوند.


Lasso، ویژگی‌ها را حذف نمی‌کند، بلکه ضرایب را به طور یکنواخت کاهش می‌دهد.

نحوه جریمه‌کردن ضرایب:

در L2 regularization (Ridge)، جریمه به صورت مجموع مربعات ضرایب به تابع هزینه اضافه می‌شود.

Ridge برای مدل‌هایی که نیاز به حفظ تمام ویژگی‌ها دارند و می‌خواهند تأثیرات ویژگی‌های کمتر مهم را کاهش دهند، مناسب است.

این تکنیک برای داده‌هایی که ویژگی‌ها همبستگی دارند، کارآمدتر است.

چگونه ترکیبی از L1 و L2 است؟

Elastic Net دو پارامتر اصلی دارد:

alpha (α) که میزان ترکیب L1 و L2 را تعیین می‌کند.

l1_ratio که نسبت تأثیر L1 و L2 را کنترل می‌کند.

اگر l1_ratio = 1، مدل Lasso خالص خواهد بود.

اگر l1_ratio = 0، مدل Ridge خالص خواهد بود.

مقادیر بین 0 و 1 ترکیبی از هر دو خواهند بود.



In [ ]:
pca =PCA(n_components=5)
X_pca_train = pca.fit_transform(X_train_scaled)
X_pca_test=pca.transform(X_test_scaled)


In [45]:
lin_pca_model = LinearRegression().fit(X_pca_train, y_train)
y_pca_test = lin_pca_model.predict(X_pca_test)
y_pca_train = lin_pca_model.predict(X_pca_train)
r2_train_pca=r2_score(y_train, y_pca_train)
r2_test_pca= r2_score(y_test, y_pca_test)
overfitting_pca= r2_train_pca - r2_test_pca
overfitting_pca




0.02380591140206878

In [46]:
overfitting_lin , overfitting_elastic, overfitting_pca 

(0.03676348536424423, 0.015997014664145937, 0.02380591140206878)

کدام مدل بهترین Test R² را دارد؟

Test R² نشان‌دهنده کیفیت پیش‌بینی‌های مدل روی داده‌های تست است. مدل‌هایی که generalize بهتری دارند، معمولاً Test R² بالاتری دارند.

با توجه به مقادیر overfitting که شما داده‌اید:

Elastic Net کمترین overfitting را دارد (0.0160)، که نشان می‌دهد generalization بهتری روی داده‌های تست داشته است.

Linear Regression (بدون regularization) بیشترین overfitting را دارد (0.0368)، که نشان می‌دهد این مدل احتمالاً عملکرد بهتری روی داده‌های آموزشی داشته ولی روی داده‌های تست ضعیف‌تر است.

PCA + Linear Regression نیز دارای overfitting کم (0.0238) است، که نشان‌دهنده عملکرد خوب آن روی داده‌های تست است.

نتیجه: مدل Elastic Net احتمالاً بهترین Test R² را دارد، زیرا به دلیل استفاده از regularization، از overfitting جلوگیری کرده است.



R²: بهتر است که R² بالای 0.8 باشد.

MSE: MSE نزدیک به 0 به معنای پیش‌بینی دقیق است.

Accuracy: Accuracy بالای 0.8 خوب است، و Accuracy 1 (100%) عالی است.